# Creating your own pdf

A core feature of zfit is the ability to create custom pdfs and functions in an simple and straightforward way.

There are two main possibilities to create a custom pdf, an easier for most use-cases and an advanced way.

[**Extended tutorial on custom PDF and code in zfit**](https://zfit-tutorials.readthedocs.io/en/master/tutorials/components/50%20-%20Custom%20code%20and%20run%20mode.html)

## The simple way

While the same works for functions, an example with a PDF is shown here.



In [ ]:
import numpy as np
import zfit
import zfit.z.numpy as znp  # numpy namespace for backend
from zfit import z

The first way is the most simple and should only be used for the trivial cases, i.e. if you're not familiar with Python classes (especially not with the `__init__` method).

In [ ]:
class MyGauss(zfit.pdf.ZPDF):
    _N_OBS = 1  # dimension, can be omitted
    _PARAMS = ['mean', 'std']  # the name of the parameters

    def _unnormalized_pdf(self, x):
        x = z.unstack_x(x)  # returns a list with the columns: do x, y, z = z.unstack_x(x) for 3D
        mean = self.params['mean']
        std = self.params['std']
        return znp.exp(- ((x - mean) / std) ** 2)

Done. Now we can use our pdf already!

The slightly more general way involves overwritting the `__init__` and gives you all the possible flexibility: to use custom parameters, to preprocess them etc.

Here we inherit from `BasePDF`

In [ ]:
class MyGauss(zfit.pdf.BasePDF):

    def __init__(self, mean, std, obs, extended=None, norm=None, name=None):
        params = {'mean': mean,  # 'mean' is the name as it will be named in the PDF, mean is just the parameter to create the PDF
                  'std': std}
        super().__init__(obs=obs, params=params, extended=extended, norm=norm,
                         name=name)

    # inside a PDF, use always `znp.something` instead of `np.something` or pure Python code
    # Otherwise, it won't be recalculated by default
    def _unnormalized_pdf(self, x):
        x = z.unstack_x(x)
        mean = self.params['mean']
        std = self.params['std']
        return znp.exp(- ((x - mean) / std) ** 2)

In [ ]:
obs = zfit.Space('obs1', limits=(-3, 6))

data_np = np.random.random(size=1000)
data = zfit.data.Data.from_numpy(array=data_np, obs=obs)

Create two parameters and an instance of your own pdf

In [ ]:
mean = zfit.Parameter("mean", 1.)
std = zfit.Parameter("std", 1.)
my_gauss = MyGauss(obs=obs, mean=mean, std=std)

In [ ]:
probs = my_gauss.pdf(data)

In [ ]:
print(probs[:20])

## Multiple dimensions and parameters with angular observables

Let's step this up!

So far, we used rather simple examples and many basic shapes, such as polynomials, already have an efficient implementation within zfit. Therefore, we will now create a three dimensional PDF measuring the angular observables of a $B^+ \rightarrow K^* l l$ decay.

The implementation is not "special" or complicated at all, it rather shows how to deal with multiple dimensions and how to manage several parameters. It was created using the equation of the angular observables (taken from a paper).

_Many thanks to Rafael Silva Coutinho for the implementation!_

In [ ]:
class AngularPDF(zfit.pdf.ZPDF):
    """Full d4Gamma/dq2dOmega for Bd -> Kst ll (l=e,mu)

    Angular distribution obtained in the total PDF (using LHCb convention JHEP 02 (2016) 104)
        i.e. the valid of the angles is given for

            - theta_l: [0, pi]
            - theta_K: [0, pi]
            - phi: [-pi, pi]
            

        The function is normalized over a finite range and therefore a PDF.

        Args:

            FL (`zfit.Parameter`): Fraction of longitudinal polarisation of the Kst
            S3 (`zfit.Parameter`): A_perp^2 - A_para^2 / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            S4 (`zfit.Parameter`): RE(A_zero*^2 * A_para^2) / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            S5 (`zfit.Parameter`): RE(A_zero*^2 * A_perp^2) / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            AFB (`zfit.Parameter`): Forward-backward asymmetry of the di-lepton system (also i.e. 3/4 * S6s)
            S7 (`zfit.Parameter`): IM(A_zero*^2 * A_para^2) / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            S8 (`zfit.Parameter`): IM(A_zero*^2 * A_perp^2) / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            S9 (`zfit.Parameter`): IM(A_perp*^2 * A_para^2) / A_zero^2 + A_para^2 + A_perp^2 (L, R)
            obs (`zfit.Space`):
            name (str):
    """

    _PARAMS = ['FL', 'S3', 'S4', 'S5', 'AFB', 'S7', 'S8', 'S9']
    _N_OBS = 3

    def _unnormalized_pdf(self, x):
        FL = self.params['FL']
        S3 = self.params['S3']
        S4 = self.params['S4']
        S5 = self.params['S5']
        AFB = self.params['AFB']
        S7 = self.params['S7']
        S8 = self.params['S8']
        S9 = self.params['S9']

        costheta_l, costheta_k, phi = z.unstack_x(x)

        sintheta_k = znp.sqrt(1.0 - costheta_k * costheta_k)
        sintheta_l = znp.sqrt(1.0 - costheta_l * costheta_l)

        sintheta_2k = (1.0 - costheta_k * costheta_k)
        sintheta_2l = (1.0 - costheta_l * costheta_l)

        sin2theta_k = (2.0 * sintheta_k * costheta_k)
        cos2theta_l = (2.0 * costheta_l * costheta_l - 1.0)
        sin2theta_l = (2.0 * sintheta_l * costheta_l)

        pdf = ((3.0 / 4.0) * (1.0 - FL) * sintheta_2k +
               FL * costheta_k * costheta_k +
               (1.0 / 4.0) * (1.0 - FL) * sintheta_2k * cos2theta_l +
               -1.0 * FL * costheta_k * costheta_k * cos2theta_l +
               S3 * sintheta_2k * sintheta_2l * znp.cos(2.0 * phi) +
               S4 * sin2theta_k * sin2theta_l * znp.cos(phi) +
               S5 * sin2theta_k * sintheta_l * znp.cos(phi) +
               (4.0 / 3.0) * AFB * sintheta_2k * costheta_l +
               S7 * sin2theta_k * sintheta_l * znp.sin(phi) +
               S8 * sin2theta_k * sin2theta_l * znp.sin(phi) +
               S9 * sintheta_2k * sintheta_2l * znp.sin(2.0 * phi))

        return pdf

### Multidimensional Spaces

This PDF now expects multidimensional data. Therefore, we need to provide a Space in multiple dimensions. The preferred way is to use the product operations to build this space from one dimensional `Space`s

In [ ]:
costhetha_k = zfit.Space('costheta_k', (-1, 1))
costhetha_l = zfit.Space('costheta_l', (-1, 1))
phi = zfit.Space('phi', (-np.pi, np.pi))
angular_obs = costhetha_k * costhetha_l * phi

### Managing parameters

Luckily, we're in Python, which provides many tools out-of-the-box. Handling parameters in a `dict` can make things very easy, even for several parameters as here.

In [ ]:
params_init = {'FL': 0.43, 'S3': -0.1, 'S4': -0.2, 'S5': -0.4, 'AFB': 0.343, 'S7': 0.001, 'S8': 0.003, 'S9': 0.002}
params = {name: zfit.Parameter(name, val, -1, 1) for name, val in params_init.items()}

In [ ]:
angular_pdf = AngularPDF(obs=angular_obs, **params)
# angular_pdf.update_integration_options(max_draws=10_000_000)
zfit.settings.set_verbosity(-1)

Extending the PDF with a dimension works easily by a product operation

In [ ]:
pdf4d = angular_pdf * my_gauss

In [ ]:
integral = angular_pdf.integrate(limits=angular_obs)  # this should be one
sample = angular_pdf.sample(n=1000)
prob_analytic = angular_pdf.pdf(sample)

In [ ]:
nll = zfit.loss.UnbinnedNLL(model=angular_pdf, data=sample)
# and fit!

In [ ]:
# takes some time due to numerical normalization of the PDF
minimizer = zfit.minimize.Minuit(
    verbosity=7,
    # gradient='zfit'
)
result = minimizer.minimize(nll)
print(result)